# 6.2 Preparing the dataset
I used the UCI Sentiment Labelled Sentences dataset to create some novel results, which is provided by by Dimitrios Kotzias, Misha Denil, Nando de Freitas, Padhraic Smyth in 2015 in the paper titled "From Group to Individual Labels Using Deep Features" published in Knowledge Discovery and Data Mining.

In [31]:
# Prepare to download the dataset
# import urllib.request
# import zipfile
# import os
# from pathlib import Path
# url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
# zip_path = "sms_spam_collection.zip"
# extracted_path = 'sms_spam_collection'
# data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"

In [32]:
# A function for downloading the dataset
# def download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path):
#     if data_file_path.exists():
#         print(f"{data_file_path} already exists. Skipping download and extraction.")
#         return

#     # Downloading the file
#     response = requests.get(url, stream=True, timeout=60)
#     response.raise_for_status()
#     with open(zip_path, "wb") as out_file:
#         for chunk in response.iter_content(chunk_size=8192):
#             if chunk:
#                 out_file.write(chunk)

#     # Unzipping the file
#     with zipfile.ZipFile(zip_path, "r") as zip_ref:
#         zip_ref.extractall(extracted_path)

#     # Add .tsv file extension
#     original_file_path = Path(extracted_path) / "SMSSpamCollection"
#     os.rename(original_file_path, data_file_path)
#     print(f"File downloaded and saved as {data_file_path}")

In [33]:
# Run the function
# download_and_unzip_sentiment_data(url, zip_path, extracted_path, data_file_path)

In [34]:
import pandas as pd
data = []
with open("sentiment_combined.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.rstrip('\n')
        text, label = line.rsplit('\t', 1)
        data.append((text, label))
df = pd.DataFrame(data, columns=["Text", "Label"])
df

,Text,Label
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
2995,The screen does get smudged easily because it ...,0
2996,What a piece of junk.. I lose more calls on th...,0
2997,Item Does Not Match Picture.,0
2998,The only thing that disappoint me is the infra...,0


In [35]:
# Examine the class label distribution
print(df["Label"].value_counts())

Label
1    1500
0    1500
Name: count, dtype: int64


In [36]:
# A function for creating a balanced dataset
# def create_balanced_dataset(df):
#     num_negative = df[df["Label"] == 0].shape[0]
#     ham_subset = df[df["Label"] == 1].sample(num_negative, random_state = 123)
#     balanced_df = pd.concat([ham_subset, df[df["Label"] == 0.0]])
#     return balanced_df

In [37]:
# # Create a balanced dataset and convert class labels
# balanced_df = create_balanced_dataset(df)
# print(balanced_df["Label"].value_counts())

In [38]:
# Swap columns
columns_titles = ["Label","Text"]
df = df.reindex(columns = columns_titles)
print(df["Label"].value_counts())

Label
1    1500
0    1500
Name: count, dtype: int64


In [39]:
# A function for splitting the dataset
def random_split(df, train_frac, validation_frac):
    df = df.sample(frac = 1, random_state = 123).reset_index(drop = True)
    train_end = int(len(df) * train_frac)
    validation_end = train_end + int(len(df) * validation_frac)
    train_df = df[:train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]
    return train_df, validation_df, test_df

In [40]:
# Split the dataset and save the parts to CSV
train_df, validation_df, test_df = random_split(df, 0.7, 0.1)
train_df.to_csv("train.csv", index = None)
validation_df.to_csv("validation.csv", index = None)
test_df.to_csv("test.csv", index = None)

# 6.3 Creating data loaders

In [41]:
# Load the tokenizer and add special padding token
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
print(tokenizer.encode("<|endoftext|>", allowed_special = {"<|endoftext|>"}))

[50256]


In [42]:
# Setting up a PyTorch Dataset class
import torch
from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=None, pad_token_id=50256):
        self.data = pd.read_csv(csv_file)
        self.encoded_texts = [tokenizer.encode(text) for text in self.data["Text"]]

        if max_length is None:
            self.max_length = self._longest_encoded_length()
        else:
            self.max_length = max_length
            self.encoded_texts = [encoded_text[:self.max_length] for encoded_text in self.encoded_texts]

        self.encoded_texts = [encoded_text + [pad_token_id] * (self.max_length - len(encoded_text)) for encoded_text in self.encoded_texts]

    def __getitem__(self, index):
        encoded = self.encoded_texts[index]
        label = self.data.iloc[index]["Label"]
        return (torch.tensor(encoded, dtype=torch.long), torch.tensor(label, dtype=torch.long))

    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        max_length = 0
        for encoded_text in self.encoded_texts:
            encoded_length = len(encoded_text)
            if encoded_length > max_length:
                max_length = encoded_length
        return max_length

In [43]:
# Load the train data with the SpamDataset class
train_dataset = SentimentDataset(csv_file = "train.csv", max_length = None, tokenizer = tokenizer)
print("The longest length is", train_dataset.max_length)

The longest length is 103


In [44]:
# Load the validation and test set
val_dataset = SentimentDataset(csv_file = "validation.csv", max_length = train_dataset.max_length, tokenizer = tokenizer)
test_dataset = SentimentDataset(csv_file = "test.csv", max_length = train_dataset.max_length, tokenizer = tokenizer)

## Exercise 6.1 - Page 179
This exercise is finished by the end of this chapter.

In [45]:
# Creating PyTorch data loaders
from torch.utils.data import DataLoader
num_workers = 0
batch_size = 8
torch.manual_seed(123)

train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True,
                          num_workers = num_workers, drop_last = True)
val_loader = DataLoader(dataset = val_dataset, batch_size = batch_size,
                        num_workers = num_workers, drop_last = False)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size,
                        num_workers = num_workers, drop_last = False)

In [46]:
# Print the tensor dimensions
for input_batch, target_batch in train_loader:
    pass
print("Input batch dimensions:", input_batch.shape)
print("Label batch dimensions:", target_batch.shape)

Input batch dimensions: torch.Size([8, 103])
Label batch dimensions: torch.Size([8])


In [47]:
# Print the total number of batches in each dataset
print(f"{len(train_loader)} training batches")
print(f"{len(val_loader)} validation batches")
print(f"{len(test_loader)} test batches")

262 training batches
38 validation batches
75 test batches


# 6.4 Initializing a model with pretrained weights

In [48]:
# Employ the same configurations as before
CHOOSE_MODEL = "gpt2-small (124M)"
INPUT_PROMPT = "So far, I had"
BASE_CONFIG = {
    "vocab_size": 50257,
    "context_length": 1024,
    "drop_rate": 0.0,
    "qkv_bias": True
}
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}
BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

In [49]:
# Loading a pretrained GPT model
from gpt_download import download_and_load_gpt2
from previous_chapters import GPTModel, load_weights_into_gpt
model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
settings, params = download_and_load_gpt2(model_size = model_size, models_dir = "gpt2")
model = GPTModel(BASE_CONFIG)
load_weights_into_gpt(model, params)
model.eval();

File already exists and is up-to-date: gpt2/124M/checkpoint
File already exists and is up-to-date: gpt2/124M/encoder.json
File already exists and is up-to-date: gpt2/124M/hparams.json
File already exists and is up-to-date: gpt2/124M/model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2/124M/model.ckpt.index
File already exists and is up-to-date: gpt2/124M/model.ckpt.meta
File already exists and is up-to-date: gpt2/124M/vocab.bpe


In [50]:
# Ensure the model generates coherent text
from previous_chapters import generate_text_simple, text_to_token_ids, token_ids_to_text
text_1 = "So far, I had"
token_ids = generate_text_simple(model = model, idx = text_to_token_ids(text_1, tokenizer), 
                                 max_new_tokens = 15, context_size = BASE_CONFIG["context_length"])
print(token_ids_to_text(token_ids, tokenizer))

So far, I had no idea what to expect.

I was so excited to see the


In [51]:
# Test model classifying capabilities
text_2 = ("Is the following text 'spam'? Answer with 'yes' or 'no': "
"'You are a winner you have been specially selected to receive $1000 cash or a $2000 award.'")
token_ids = generate_text_simple(model=model, idx=text_to_token_ids(text_2, tokenizer),
                                 max_new_tokens=23, context_size=BASE_CONFIG["context_length"])
print(token_ids_to_text(token_ids, tokenizer))

Is the following text 'spam'? Answer with 'yes' or 'no': 'You are a winner you have been specially selected to receive $1000 cash or a $2000 award.'

The following text 'spam'? Answer with 'yes' or 'no': 'You are a winner


# 6.5 Adding a classification head

In [ ]:
# Print the model architecture
print(model)

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.0, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_resid): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768,